### Data collection for various metrics:

Data Collection for Safety and Stability Metrics (Ignore the efficiency metrics even though they are displayed). 

Collection performed in 3 stages

    A. Data collection for Safety and CAV (Controller Acceleration Variation) 
    B. Data collevtion for WAR (Wave Attenuation Ratio)
    C. Data collection for Stability Plot

### Notes:


- Desired velocity change for FS May need monitoring (May not be set for all possible lengths)
- PI can have failures, so have to turn ON render and manually check
- LACC needs a different shock time (for no shock change time at eval)
- Generated data will be saved in the folder `test_time_rollout`
- We apply shocks after the systems have been stabilized
- Default shock times are between 8000 and 11000 unless explicitly specified otherwise
- Difference: For stability plot, the length is 270 whereas for WAR, the length is 260
- For WAR num_rollours is set to 1

### Penetration rates
- num_controlled = 1 (5%), 4(20%), 9(40%), 13(16%) 

### Changes that need to be made to files: 

    1. 
    2. For Singleagent, Efficiency at test time, make the accel behavior zero from apply_rl_actions in density_aware_env (single agent version)
    3. For Multiagent, Efficiency at train time. Make sure the leader is correct in multiagent_ring. Make sure all test time markers are turned off. Make sure to have correct number of NUM_AUTOMATED at 2 places
    4. For Multiagent, Efficiency at test time, make sure the correct exp_configs are present in the same folder as test_rllib.py. in controllers for daware, set efficiency to True for TrainedAgent. 
    5. For Multiagent, Safety + Stability at test time, followers zero acceleration behavior is set from apply_rl_actions in density_aware_env (multiagent version)


### A and B together: (Data collection for Safety and CAV) + (Data collection for WAR)

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260


### 1. (100%) IDM 

In [ ]:
!python classic.py --method idm --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render

In [ ]:
!python eval_metrics.py --method idm --save_plots

In [ ]:
# IDM does not need WAR, un-applicable

### 2. (5%) Single Vehicle Control

__FS__

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

In [ ]:
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__
- This requires verifying if the particular run was `SUCCESS` or `FAIL`
- i.e., the controller may fail to stabilize the system

In [ ]:
# For CAV (first: get the data)
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

# Stable or not before shocks were applied
# 1. Stable 
# 2. Stable
# 3. Stable
# 4. Stable
# 5. Stable


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

# Stable or not before shocks were applied
# 1. 
# 2. 
# 3. 
# 4. 
# 5. 


In [ ]:
!python eval_plots.py --method pi 

#### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# No WAR. BCM is Unstable. But useful for later, to estimate in bottleneck.
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled 1

In [ ]:
!python eval_plots.py --method bcm 

#### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --shock_start_time 11400 --shock_end_time 15000 --num_controlled 1


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

In [ ]:
# No WAR. LACC is Unstable. But useful for later, to estimate in bottleneck.
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 1 --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

__Wu et. al.__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50 \
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method wu 

__Ours (1x)__
- The local zone is set to 50m at training. And it is set to 50m here as well.

In [ ]:
# For CAV (Safety + Stability)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [ ]:
# Only look at safety and stability
!python eval_metrics.py --method ours --save_plots

In [ ]:
# For CAV (Efficiency)

!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_6106dcf6_2024-02-08_12-42-07_7vnf1n4 200\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# Only look at Efficiency (Wipe out the previous data)
!python eval_metrics.py --method ours --save_plots

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### 3. (20%) Multiple Vehicle Control

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts\
$NUM_ROLLOUTS --shock --render --num_controlled 4


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR. Desired velocity should be set to 4.82 here
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 4


In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__

In [ ]:
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS\
--shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 4


In [ ]:
!python eval_plots.py --method pi 

__BCM__

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method bcm 

__LACC__

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts 5 --gen_emission\
--shock --render --num_controlled 4 --shock_start_time 11400 --shock_end_time 15000


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

In [ ]:
# Unstable so no WAR

__Wu et. al. (20%)__

In [ ]:
# Agents are supposed to be dispersed.
# Stabalization time looks high, shocks can start later
!python test_rllib.py ./Wu_et_al/Trained_policies/20_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_436e49a2_2024-02-12_13-33-49wdl9scs5 174\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Wu_et_al/Trained_policies/20_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_436e49a2_2024-02-12_13-33-49wdl9scs5 174\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method wu

__Ours (20%)__

- The local zone of __leader__ is set to 50m at training. It is kept at 50m here as well.

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
# Set efficiency to True in TrainedAgentController

!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_33361ae0_2024-02-11_20-29-43f1pzfgrz 140\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
# Data for WAR (taken on the Safety + Stability model). Make sure everything is tuned for Safety + Stability
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method ours4x

### 4. (40%) Multiple Vehicle Control

- 9 vehicles out of 22

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260
NUM_CONTROLLED = 9

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR Desired velocity should be set to 4.82 here
!python classic.py --method fs --stability --length $LENGTH\
--num_rollouts 5 --gen_emission --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method fs 

### PIwS

In [ ]:
!python classic.py --method piws --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method pi 

### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --num_controlled $NUM_CONTROLLED


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots


In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method bcm 

### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots


In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Wu et. al. (40%)

In [ ]:
# Agents are supposed to be dispersed.
# Stabalization time looks high, shocks can start later
!python test_rllib.py ./Wu_et_al/Trained_policies/40_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_b8f3c414_2024-02-14_13-13-15_l2ifa89 92\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 9


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR

In [ ]:
# WAR

### Ours (40%)

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_MultiAgentDensityAwareRLEnv-v1_589d08ce_2024-02-14_13-03-242t0mb06j 60\
--method ours9x --length $LENGTH --num_rollouts 20 --gen_emission --shock --render --num_controlled $NUM_CONTROLLED



In [1]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours9x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351531711561913.8572984-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351541711561914.870917-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351571711561917.4615736-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357031711562223.613163-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357071711562227.076018-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357091711562229.6841137-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1402281711562548.3451078-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1402301711562550.506555-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1402301711562550.9

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.03747761 0.03005051 0.06684671 0.02458007 0.03536008 0.04751295
 0.03896835 0.05099891 1.02786386]

DRAC (m/s^2):
	Worst= 1.03

(9,)
Worst case time to collision for each vehicle (s): 
[-11.95741566 -11.37042232 -13.68338574 -11.32881106  -9.49604165
 -11.7365469   -9.22772221 -23.6541705   -8.63004677]

Best case time to collision for each vehicle (s): 
[-120848.00042326  -35854.73101245  -23942.44144639  -27214.83688566
  -33826.32586327  -31263.46638641 -757008.59267182  -27546.5253231
  -28707.54094134]

Average time to collision for each vehicle (s): 
[-281.44513261 -223.43977374 -229.37286811 -242.10412557 -200.97892408
 -269.00725926 -730.70110044 -288.14821736 -337.67570484]

Standard deviation of time to collision for each vehicle (s): 
[ 2980.71195481  1335.66633131  1088.43183515  1238.5391875
   977.24652834  1405.28583964 17829.25707031  1213.63247382
  1524.31090647]

Time to Collision (s):
	Worst= -8.63
	Best= -757008.59

(22, 3600)

Average speed of each vehicle (m/s): 
[3.89207595 3.89274177 3.89314485 3.89314868 3.89169005 3.89121427
 3.89118935 3.89145931 3.89067448 3.88986808 3.89036592 3.89076273
 3.89101439 3.89130705 3.89114049 3.89107775 3.89100006 3.89101914
 3.89122055 3.89158899 3.89176611 3.89193441]

Standard deviation of speed of each vehicle (m/s): 
[0.25032118 0.24906796 0.24667598 0.24471433 0.24342373 0.24191847
 0.23942651 0.23814988 0.22957543 1.54455165 1.43466109 1.37689036
 1.3322466  1.25283239 1.13824196 1.10095956 1.05414019 0.99771061
 0.9022989  0.70689398 0.6918975  0.49160866]

Speed of all vehicles (m/s): Average= 3.89, std=0.74

####################
Total number of vehicles that crossed the zero point: 117

Flow (veh/hour): 1170.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[11.43467554 11.39139814 11.31487039 11.37794116 11.4863799  11.36411645
 11.17257249 11.14840659  9.82199528 12.94382189 12.62907508 12.55480063
 12.33163261 12.0696591

####################
Acceleration variation (m/s^2): [0.41090701701068477]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1351571711561917.4615736-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.01524211 1.01330503 1.00884284 1.0068348  1.00416533 1.00314379
 1.00103791 1.00179508 1.02925285]


Average time headway of controller(s): 
[ 1.0790987   1.07617067  1.06417906  1.06721226  1.07307367  1.07470102
  1.06684119  1.06645687 15.1925925 ]

Standard deviation of time headway of controller(s): 
[0.04292271 0.03999251 0.02783034 0.03362281 0.03584039 0.04045943
 0.03232542 0.03392035 8.43078861]

Time headway (s): Worst = 1.0, Avg= 2.64, std= 0.97

####################
####################
Acceleration variation (m/s^2): [0.41090701701068477, 0.42438471943969913]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240327-1357031711562223.613163-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.01252224

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_MultiAgentDensityAwareRLEnv-v1_92538b0c_2024-02-15_14-36-53fn6l9uvs 140\
--method ours9x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --render --shock --num_controlled $NUM_CONTROLLED


In [2]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours9x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1332271711647147.9690323-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1332291711647149.3188415-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.2143638-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.4356213-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1342511711647771.5272796-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1342521711647772.4146538-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1347551711648075.8010864-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1347571711648077.8549862-0_emission.csv', './test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-13525817116483

(9,)
Worst case drac for each vehicle (m/s^2): 
[0.0712485  0.05100663 0.06144022 0.04875027 0.0628692  0.06698184
 0.07764625 0.1121422  0.78910665]

DRAC (m/s^2):
	Worst= 0.79

(9,)
Worst case time to collision for each vehicle (s): 
[-2.04510937 -3.90059853 -3.93497257 -3.49004429 -3.54930733 -3.32856855
 -2.98594429 -2.58259132 -1.64771624]

Best case time to collision for each vehicle (s): 
[-93289.49145805 -11605.15725012 -32943.1739     -12418.05074794
 -31540.12735257 -12719.2502473  -52162.81292689 -59974.62874733
  -5707.2812551 ]

Average time to collision for each vehicle (s): 
[-212.20985368 -174.01258412 -215.38320621 -171.96574795 -242.23586529
 -190.41967746 -349.15487023 -339.41344071  -48.60920546]

Standard deviation of time to collision for each vehicle (s): 
[2468.59297267  650.34365424 1233.61993726  658.89213193 1330.34963569
  637.00243893 2007.40063692 1963.97842283  195.40460378]

Time to Collision (s):
	Worst= -1.65
	Best= -93289.49
	Average= -215.93
	Std= 12

(22, 3600)

Average speed of each vehicle (m/s): 
[4.44346512 4.44324177 4.44293657 4.44238836 4.44238998 4.44231235
 4.44185196 4.44262394 4.44213279 4.45419877 4.45267903 4.45168402
 4.45111899 4.45053116 4.45011778 4.44935721 4.44889188 4.44825066
 4.44730637 4.44643081 4.44517794 4.4440115 ]

Standard deviation of speed of each vehicle (m/s): 
[1.41977381 1.37045331 1.26624554 1.25935673 1.27420715 1.28121912
 1.28667919 1.29920351 1.2995497  2.28714293 2.2761318  2.26809284
 2.23417767 2.18658328 2.13312659 2.07607496 1.99843891 1.91950613
 1.86415016 1.77048482 1.73174457 1.66270583]

Speed of all vehicles (m/s): Average= 4.45, std=1.73

####################
Total number of vehicles that crossed the zero point: 135

Flow (veh/hour): 1350.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.64355954 13.66363328 13.28730844 13.43174989 13.26365099 13.2923708
 13.17290852 13.05310968 12.76006476 13.28074635 13.50491038 13.62283597
 13.62450903 13.9235848 

####################
Acceleration variation (m/s^2): [0.49056660665880786]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.2143638-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.00897294 1.01213864 1.0034282  1.00194048 0.99791805 0.99452632
 0.98972657 0.98186268 1.00321703]


Average time headway of controller(s): 
[1.10241739 1.13897631 1.07254095 1.06033185 1.06086589 1.0555462
 1.05017719 1.06376677 2.96456761]

Standard deviation of time headway of controller(s): 
[0.13663462 0.4363968  0.09839923 0.03054454 0.05621011 0.03258768
 0.0309859  0.0656992  2.2935494 ]

Time headway (s): Worst = 0.98, Avg= 1.29, std= 0.35

####################
####################
Acceleration variation (m/s^2): [0.49056660665880786, 0.483001306313162]

File: ./test_time_rollout/ours9x/density_aware_multiagent_ring_20240328-1337431711647463.4356213-0_emission.csv
(9, 3600)
Worst case time headway of controller(s): 
[1.00899187 1.0126214

In [ ]:
# Data for WAR

In [ ]:
# WAR

### 5. (60%) Multiple Vehicle Control
- 13 vehicles controlled

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260
NUM_CONTROLLED = 13

### FS

In [ ]:
# Change get_desired_velocity velocity to 4.55 instead of 4.82
# Still it can get completely destabilitzed
!python classic.py --method fs --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED

In [ ]:
# use the eval_metrics code. May have to make 'fs' in tts calculation
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR. Change the desired velocity to 4.55
!python classic.py --method fs --stability --length $LENGTH\
--num_rollouts 5 --gen_emission --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method fs 

### PIwS

In [ ]:
!python classic.py --method piws --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method pi 

### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots


In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method bcm 

### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots


In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Wu et. al. (60%)

In [ ]:
# Agents are supposed to be dispersed. Exp configs folder should be present outside where test_rllib.py is present
# Stabalization time looks high, shocks can start later
# 90 is highest but cannot stabilize at 90
!python test_rllib.py ./Wu_et_al/Trained_policies/60_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_24561808_2024-02-15_12-53-36bmm0ybn0 130\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


# !python test_rllib.py ./Wu_et_al/Trained_policies/60_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_31346110_2024-02-15_12-53-57as6iehau 90\
# --method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED



In [ ]:
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR

In [ ]:
# WAR

### Ours (60%)

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/60_percent/PPO_MultiAgentDensityAwareRLEnv-v1_18eaa6f2_2024-02-16_15-29-348o30nqxq 200\
--method ours13x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method ours13x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/60_percent/PPO_MultiAgentDensityAwareRLEnv-v1_ed546dfe_2024-02-14_17-18-06nijs5agx 200\
--method ours13x --length $LENGTH --num_rollouts 20 --render --gen_emission --shock --num_controlled $NUM_CONTROLLED


In [3]:
!python eval_metrics.py --method ours13x --save_plots

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1406551711649215.6147149-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1406571711649217.6614523-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1412321711649552.7959552-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1412381711649558.5798147-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1418181711649898.3592672-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1418231711649903.291005-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1424081711650248.655149-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1424121711650252.4301355-0_emission.csv', './test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1430121

(13,)
Worst case time to collision for each vehicle (s): 
[-6.49117738 -5.95076901 -6.38742904 -6.47713765 -6.4106628  -5.78455977
 -4.75848648 -5.52647928 -5.2537461  -4.42245083 -3.9614063  -2.73399937
 -1.18525872]

Best case time to collision for each vehicle (s): 
[ -66891.79062769  -27817.15408823  -16576.87583911 -224509.05595205
  -17408.73474597  -12542.93427568  -26089.77364981  -48301.95175804
  -16947.16252596 -553348.82598391 -763796.49850691  -73743.6528025
  -51426.62560411]

Average time to collision for each vehicle (s): 
[-251.83269913 -248.89287069 -190.56056659 -366.56447233 -201.22239602
 -192.88272073 -266.88857827 -275.95340697 -215.19312652 -516.54793164
 -762.32135496 -297.15819095 -220.31417655]

Standard deviation of time to collision for each vehicle (s): 
[ 1945.09636104  1296.96560613   797.19649425  5514.85742632
   737.29001848   605.72247144  1222.50692745  1606.47288232
   865.31009042 13153.45591023 18440.18966638  2088.04730036
  1429.86561486]

Time

(22, 3600)

Average speed of each vehicle (m/s): 
[4.85983956 4.85721574 4.85445441 4.85219836 4.84929262 4.84702965
 4.84448779 4.8419407  4.83963562 4.83711333 4.83491166 4.8324541
 4.82977793 4.87225442 4.87182503 4.87120634 4.87017206 4.86853753
 4.86664779 4.86444461 4.86254067 4.86096702]

Standard deviation of speed of each vehicle (m/s): 
[0.3029172  0.29296126 0.29113855 0.28760219 0.28724806 0.28610602
 0.28661982 0.28836244 0.29106235 0.29230078 0.2918005  0.2882483
 0.27330316 1.29748411 1.24176494 1.15102897 1.13259277 0.9672671
 0.89938897 0.85949459 0.68634618 0.46301415]

Speed of all vehicles (m/s): Average= 4.85, std=0.57

####################
Total number of vehicles that crossed the zero point: 148

Flow (veh/hour): 1480.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[14.14699643 14.03684273 14.07813251 13.95699254 13.82552615 13.61304796
 13.455019   13.48216274 13.69829747 13.90059108 14.13572235 13.93921742
 12.56072604 15.43776259 1

(22, 3600)

Average speed of each vehicle (m/s): 
[4.7329076  4.73074062 4.72872337 4.72523964 4.72185599 4.71858483
 4.71422591 4.71125916 4.70850349 4.70599133 4.70311274 4.69912849
 4.69607089 4.75628183 4.75341084 4.75109284 4.74873353 4.74608552
 4.74358761 4.74127272 4.7386158  4.73567947]

Standard deviation of speed of each vehicle (m/s): 
[0.4573594  0.45843378 0.46446963 0.46574161 0.4668121  0.4657872
 0.46607608 0.48167182 0.48088162 0.48494701 0.49331544 0.49193615
 0.46993585 1.56208065 1.4095991  1.34361734 1.27954452 1.11245284
 0.91128506 0.89993644 0.87251243 0.57693536]

Speed of all vehicles (m/s): Average= 4.73, std=0.73

####################
Total number of vehicles that crossed the zero point: 143

Flow (veh/hour): 1430.0

####################
(22, 3600)

Miles per gallon for each vehicle (mpg):
[13.94836289 13.97886862 13.78358352 13.6286634  13.54719861 13.27648002
 13.04957059 13.37432483 13.39782829 13.52420051 13.84286769 13.68198884
 12.24168371 14.32691955

####################
Acceleration variation (m/s^2): [0.5538855779235708, 0.580207302534981, 0.5967342425247396, 0.5615554460630324, 0.574394687452685, 0.5707020221182018]

File: ./test_time_rollout/ours13x/density_aware_multiagent_ring_20240328-1424121711650252.4301355-0_emission.csv
(13, 3600)
Worst case time headway of controller(s): 
[1.0087471  1.01385851 1.00867075 1.00643521 1.00732789 1.00602349
 1.0070935  1.01007115 0.99883373 0.99738125 0.99769603 0.99784968
 0.99657888]


Average time headway of controller(s): 
[1.09671449 1.0772338  1.08635707 1.08181067 1.11015896 1.08453375
 1.11130366 1.08682734 1.08145379 1.07184621 1.06859047 1.06764115
 4.04284448]

Standard deviation of time headway of controller(s): 
[0.05662194 0.04487983 0.06684946 0.12202146 0.42445752 0.12228142
 0.40556168 0.21950568 0.19092499 0.09437185 0.03886424 0.03756981
 1.87788801]

Time headway (s): Worst = 1.0, Avg= 1.31, std= 0.28

####################
####################
Acceleration variation (m/

In [ ]:
# Data for WAR

In [ ]:
# WAR

## C. Data collection for Stability Plot
- Execute the `eval_plots.py` code as well because this will create the stability plots (sometimes manual inspection is required)

In [ ]:
LENGTH_STABILITY = 270 #m
NUM_ROLLOUTS_STABILITY = 1

### IDM

In [ ]:
# This is for plot
# Stability shock is applied to IDM when the system is stable and SGT is not formed yet
# 150 is pretty early-on
!python classic.py --method idm --length $LENGTH_STABILITY --gen_emission --stability --shock_start_time 150 --shock_end_time 3710 --render

In [ ]:
# IDM is unstable (So there is no wave attenuation) - WAR does not apply
# But do we still need a value for reference (WAR = 0)
!python eval_plots.py --method idm --start_time 150 --end_time 3710 

### FS

In [ ]:
!python classic.py --method fs --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render

In [ ]:
!python eval_plots.py --method fs 

### PIwS

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render
# Check if was stable or not before the shock was applied

In [ ]:
!python eval_plots.py --method pi 

### RL+ LO (Wu et al.)

In [ ]:
!python test_rllib.py ./Wu_et_al/Trained_policies/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_STABILITY --render


In [ ]:
!python eval_plots.py --method wu 

### Ours (5%)
- Set local zone to 35m in the file before executing the code below

In [ ]:
# This data is taken with the local zone set to 35m

# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### BCM

In [ ]:
!python classic.py --method bcm --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render


In [ ]:
!python eval_plots.py --method bcm 

### Ours (20%)
- Set Local zone to 40m in the file before executing the code below 

In [ ]:
# This data is taken with the local zone set to 40m
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
# Data for WAR (taken on the Safety + Stability model). Make sure everything is tuned for Safety + Stability

!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method ours4x

### LACC

In [ ]:
!python classic.py --method lacc --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --shock_start_time 11400 --shock_end_time 15000 --render

In [ ]:
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200